In [ ]:
import json
from queue import Queue

In [ ]:
with open("intermediate_outputs/formatted_appState_w_dependencies.json", "r") as f:
    data = json.load(f)

In [ ]:
def build_linked_path(name: str):
    visited_linked_path = []; # to prevent infinite recursion
    def helper(_name: str):
        global data;
        
        # eliminate widgets that have no dependants
        if _name not in data or (
            _name != name
            and "Widget" in data[_name]["subtype"] 
            and len(data[_name]["component_relations"].get("dependants", [])) == 0
        ):
            return None
        
        if _name in visited_linked_path:
            return {
                "name": _name,
                "type": "RECURSIVE",
                "chained_events": []
            }
        
        visited_linked_path.append(_name)

        # truncate manual triggers
        if _name != name and (
            any(subtype in data[_name]["subtype"] for subtype in ["Button", "Select"])
            or (
                data[_name]["type"] == "datasource"
                and any(action in _name for action in ["create", "update", "delete", "add", "remove"])
            ) 
        ):
            return {
                "name": _name,
                "type": data[_name]["type"],
                "chained_events": []
            }

        return {
            "name": _name,
            "type": data[_name]["type"],
            "component_relations": data[_name]["component_relations"],
            "query": data[_name].get("query"),
            "chained_events": [
                event 
                for event 
                in [
                    helper(child["pluginId"]) 
                    for child 
                    in data[_name]["component_relations"].get("dependants", [])
                ]
                if event is not None
            ]
        }
    return helper(name)

def get_recursive_name(linked_path_node: dict):
    visited_recursive_name = []; # to prevent infinite recursion
    def helper(_linked_path_node: dict):
        if _linked_path_node["name"] in visited_recursive_name:
            return {
                "name": _linked_path_node["name"],
                "chained_events": []
            }
        visited_recursive_name.append(_linked_path_node["name"])
        return {
            "name": _linked_path_node["name"],
            "chained_events": [get_recursive_name(event) for event in _linked_path_node["chained_events"]]
        }
    return helper(linked_path_node)

def get_names_flat_list(names: dict):
    flat_list = []
    def helper(_names: dict):
        flat_list.append(_names["name"])
        for event in _names["chained_events"]:
            helper(event)
    helper(names)
    return list(set(flat_list))

In [ ]:
# get first key in data
first_key = list(data.keys())[3]
first_key

In [ ]:
[
    key 
    for key 
    in data.keys() 
    if data[key]["type"] in ["datasource", "function"] 
        or any(
            subtype in data[key]["subtype"] 
            for subtype 
            in ["Button", "Input", "Select"]
        )
]

In [ ]:
linked_paths = [
    build_linked_path(key) 
    for key 
    in data.keys() 
    if data[key]["type"] in ["datasource", "function"] 
        or any(
            subtype in data[key]["subtype"] 
            for subtype 
            in ["Button", "Input", "Select"]
        )
]
print("selectServiceLocation" in linked_paths)
names_linked_lists = [get_recursive_name(linked_path) for linked_path in linked_paths]
names_flat_lists = [get_names_flat_list(linked_path) for linked_path in linked_paths]
names_flat_lists

# generate dot file

In [ ]:
def get_list_of_links(linked_path: dict):
    links = []
    def helper(_linked_path: dict):
        for event in _linked_path["chained_events"]:
            links.append({
                "source": _linked_path["name"],
                "target": event["name"]
            })
            helper(event)
    helper(linked_path)
    return sorted(links, key=lambda link: link["source"])

events_lists = [{"index": index, "title": lp.get("name"), "links": get_list_of_links(lp)} for [index, lp] in enumerate(linked_paths)]
events_lists = [list for list in events_lists if len(list.get("links")) > 0]
events_lists

In [ ]:
def build_digraph(title: str, events_lists: list[dict]):
    nodes = {}
    for [index, name] in enumerate(list(set([name for el in events_lists for name in get_names_flat_list(linked_paths[el["index"]])]))):
        nodes[name] = index

    def build_subgraph(label: str, links: list[str], color: str = "black"):
        return f"""
        subgraph cluster_{label} {{
        \tstyle="rounded";
        \tlabel="{label}";
        \tcolor="{color}";
        \tpenwidth=3;\n""" + \
        "\n".join([
            f"\t\t{nodes.get(link['source'])} -> {nodes.get(link['target'])} [ color=\"{color}\", penwidth=3 ];" 
            for link 
            in links
        ]) + \
        "\n\t}"


    dot = f"""digraph {title} {{
        rankdir=LR;
        node [shape=record, style=filled];
        edge [color=black]; 
    """

    for [label, index] in nodes.items():
        fillcolor = "gray90";
        color = "black";
        _type = None
        _subtype = None
        if data.get(label):
            _type = data.get(label).get("type")
            _subtype = data.get(label).get("subtype")
            if _type == "datasource":
                fillcolor = "#BBD686"
                if any(action in label for action in ["create", "update", "delete", "add", "remove"]):
                    fillcolor = "red4"
                    color = "white"
            if _type == "function":
                fillcolor = "cornflowerblue"
                color = "white"
            elif any(subtype in _subtype for subtype in ["Button", "Select"]):
                fillcolor = "red4"
                color = "white"
            elif "Table" in _subtype:
                fillcolor = "darkgreen"
                color = "white"
            elif "Input" in _subtype:
                fillcolor = "lightpink"
            elif any(subtype in _subtype for subtype in ["Container", "Icon", "Text"]):
                fillcolor = "white"
        if label == title:
            fillcolor = "black";
            color = "white";
        
        dot += f'\t{index} [label="{{ {label}|{{ {_type}|{_subtype} }} }}" fillcolor="{fillcolor}" fontcolor="{color}"];\n'

    for events_list in events_lists:
        dot += build_subgraph(events_list.get("title"), events_list.get("links"))

    dot += "}"
    folder_path = f"outputs/traversals"
    if len(nodes.keys()) > 20:
        folder_path = f"outputs/problematic_traversals"
    with open(f"{folder_path}/{title}.dot", "w") as f:
        f.write(dot)

# Clean Output Folders

In [ ]:
import shutil
import os
try:
    for path in ["traversals", "problematic_traversals"]:
        if os.path.exists(f"outputs/{path}"):
            shutil.rmtree(f"outputs/{path}")
        os.mkdir(f"outputs/{path}")
    print("wiped folders")
except OSError as e:
    print(f"Error: {e}")

In [ ]:
# build_digraph("full_graph", events_lists)

In [ ]:
for events_list in events_lists:
    build_digraph(events_list.get("title"), [events_list])